In [5]:
import pandas as pd
import os
import re
import csv

In [11]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\minh.khuatduynhat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
input_folder = 'EN'  # Update with your folder path
output_folder = 'output'  # Folder to save extracted sections
# List of section names to look for (in order)
section_names = [
    'PERSONAL DETAIL',
    'ABOUT',
    'EDUCATIONAL BACKGROUND',
    'CERTIFICATION',
    'LANGUAGE',
    'TECHNICAL SKILL',
    'EXPERIENCE'
]

In [3]:
def remove_accent(text):
    """Removes Vietnamese accents from a string."""
    text = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', text)
    text = re.sub(r'[ÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴ]', 'A', text)
    text = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', text)
    text = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', text)
    text = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', text)
    text = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', text)
    text = re.sub(r'[ìíịỉĩ]', 'i', text)
    text = re.sub(r'[ÌÍỊỈĨ]', 'I', text)
    text = re.sub(r'[ùúụủũưừứựửữ]', 'u', text)
    text = re.sub(r'[ÙÚỤỦŨƯỪỨỰỬỮ]', 'U', text)
    text = re.sub(r'[ỳýỵỷỹ]', 'y', text)
    text = re.sub(r'[ỲÝỴỶỸ]', 'Y', text)
    text = re.sub(r'[đ]', 'd', text)
    text = re.sub(r'[Đ]', 'D', text)
    return text

In [4]:
# Function to find the start row of a section
def find_section_row(section_name):
  try:
      return data[data.apply(lambda x: x.str.contains(section_name, case=True, na=False)).any(axis=1)].index[0]
  except IndexError:
      return None  # Return None if the section is not found

for filename in os.listdir(input_folder):
  if filename.endswith('.xlsx'):  # Only process Excel files
    input_file = os.path.join(input_folder, filename)
    output_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_sections.txt")
  # Read the Excel file
    data = pd.read_excel(input_file, header=None)  # Read without a header since this is structured data
    data = data.astype(str)

    # Strip leading/trailing spaces
    data = data.map(lambda x: x.strip() if isinstance(x, str) else x)

    # Find start rows for all sections
    sections = {name: find_section_row(name) for name in section_names}

    # Filter out missing sections and sort by row index
    sections = {name: row for name, row in sections.items() if row is not None}
    sorted_sections = sorted(sections.items(), key=lambda x: x[1])  # Sort by start row

    # Add a pseudo end row for the last section
    end_row = find_section_row('I declare that the above information is accurate.')
    if end_row is None:
        end_row = data.shape[0]  # Default to the last row of the data if the end row is not found

    # Process each section
    output_content = []
    for i, (section_name, start_row) in enumerate(sorted_sections):
        # Determine the end row for this section
        next_start_row = sorted_sections[i + 1][1] if i + 1 < len(sorted_sections) else end_row
        section_data = data.iloc[start_row:next_start_row, :]

        # Combine rows into a single string
        section_text = section_data.apply(lambda row: ' '.join(row.dropna()), axis=1)
        section_content = '\n'.join([line.replace('nan', '').replace('- ', '').replace(' – ', '~').strip() for line in section_text if line.strip()]).strip()

        # Remove Vietnamese accents
        section_content = remove_accent(section_content)

        # Append the section content to the output
        output_content.append(f"{section_content}\n")

    # Save the output to a text file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.writelines(output_content)

    print(f"Extracted content has been saved to {output_file}")

Extracted content has been saved to output\Copy of CV-Vu-Viet-Anh_sections.txt
Extracted content has been saved to output\Copy of VTI - CV Bui Duc Anh_sections.txt
Extracted content has been saved to output\CV - TO QUY THANH_sections.txt
Extracted content has been saved to output\CV Pham Viet Hieu - Cloud SA1_sections.txt
Extracted content has been saved to output\CV-Doan-Dinh-Vu-Cong_sections.txt
Extracted content has been saved to output\CV-Le Huu Minh_sections.txt
Extracted content has been saved to output\CV-Le Sy Quang_sections.txt
Extracted content has been saved to output\CV-Ngo-Thach-Anh_sections.txt
Extracted content has been saved to output\CV-Nguyen Phi Hai Nam_sections.txt
Extracted content has been saved to output\CV-Nguyen Thi Hong Nhung_sections.txt
Extracted content has been saved to output\CV-NguyenHaiMy-EN_sections.txt
Extracted content has been saved to output\CV-NguyenQuangHoa_sections.txt
Extracted content has been saved to output\CV-Pham-Thi-Minh-Luong-PM _section

C:\Users\minh.khuatduynhat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\packaging\custom.py:213: UserWarning: Unknown type for KSOProductBuildVer
  warn(f"Unknown type for {prop.name}")


Extracted content has been saved to output\VTI_CV_Thai-Xuan-Phuong_DevOps_sections.txt


In [54]:
section_mapping = {
    "PERSONAL DETAIL": "Personal Detail",
    "PERSONAL DETAILS": "Personal Detail",
    "ABOUT": "About",
    "ABOUTS": "About",
    "EDUCATIONAL BACKGROUND": "Educational Background",
    "EDUCATION": "Educational Background",
    "LANGUAGE": "Language",
    "LANGUAGES": "Language",
    "CERTIFICATION": "Certification",
    "CERTIFICATIONS": "Certification",
    "SKILL": "Technical Skill",
    "SKILLS": "Technical Skill",
    "TECHNICAL SKILL": "Technical Skill",
    "TECHNICAL SKILLS": "Technical Skill",
    "EXPERIENCE": "Experience",
    "WORK EXPERIENCE": "Experience",
    "EXPERIENCES": "Experience",
    # Add more mappings as needed
}

In [55]:
def normalize_section_header(header):
    """
    Normalize a section header using the mapping.
    """
    header_clean = header.strip().upper()
    # Return the normalized name if found; otherwise, capitalize normally.
    return section_mapping.get(header_clean, header_clean.title())


In [56]:

def parse_line(line):
    """
    Split a line into key-value parts using tab or multiple spaces as delimiters.
    """
    if '\t' in line:
        parts = line.split('\t')
    else:
        parts = re.split(r'\s{2,}', line)
    return [part.strip() for part in parts if part.strip()]


In [57]:
with open('cv.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

data = {}
current_section = None
section_content = []

In [58]:
for line in lines:
    line = line.strip()
    if not line:
        continue

    # Check if line is a section header.
    # Either it is all uppercase (with spaces) or it matches one of our mapped keys.
    if re.fullmatch(r'[A-Z ]+', line) or line.strip().upper() in section_mapping:
        # Save content of previous section, if any.
        if current_section and section_content:
            if current_section not in data:
                data[current_section] = " ".join(section_content)
            else:
                data[current_section] += " " + " ".join(section_content)
        # Normalize the section header.
        current_section = normalize_section_header(line)
        section_content = []
        continue

    # Try parsing the line as a key-value pair.
    parts = parse_line(line)
    if len(parts) == 2:
        key, value = parts
        if current_section and section_content:
            data[current_section] = " ".join(section_content)
            current_section = line
            section_content = []
        else:
            section_content.append(line)
    else:
        section_content.append(line)

In [59]:
if current_section and section_content:
    if current_section not in data:
        data[current_section] = " ".join(section_content)
    else:
        data[current_section] += " " + " ".join(section_content)

In [60]:
flat_data = {}
for key, value in data.items():
    if isinstance(value, dict):
        for sub_key, sub_value in value.items():
            flat_data[f"{key}_{sub_key}"] = sub_value
    elif isinstance(value, list):
        flat_data[key] = "; ".join(value)
    else:
        flat_data[key] = value


In [61]:
with open('cv.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=flat_data.keys())
    writer.writeheader()
    writer.writerow(flat_data)

print("CSV file 'cv.csv' created successfully!")

CSV file 'cv.csv' created successfully!


In [75]:
import re
import csv

# Define mapping from text file headers to CSV column names
section_headers = {
    'PERSONAL DETAIL': 'Personal detail',
    'ABOUT': 'About',
    'EDUCATIONAL BACKGROUND': 'Education',
    'LANGUAGE': 'Language',
    'TECHNICAL SKILL': 'Technical Skill',
    'EXPERIENCE': 'Experience'
}

# Read the text file and clean up the lines
with open('input.txt', 'r', encoding='utf-8') as f:
    lines = [line.strip() for line in f if line.strip()]

# Prepare a dictionary to hold each section’s lines
data = {col: [] for col in section_headers.values()}
current_section = None

# Process lines and group them by their section
for line in lines:
    if line in section_headers:
        current_section = section_headers[line]
    else:
        if current_section:
            data[current_section].append(line)

# Function to process key–value lines: splits on 2+ spaces and joins with a colon.
def process_key_value(lines, join_sep=';'):
    processed = []
    for line in lines:
        parts = re.split(r'\s{2,}', line)
        if len(parts) >= 2:
            processed.append(f"{parts[0]}:{parts[1]}")
        else:
            processed.append(parts[0])
    return join_sep.join(processed)

# Process each section as needed
result = {}
result['Personal detail'] = process_key_value(data['Personal detail'], join_sep=';')
# For the ABOUT section, join all lines with a space
result['About'] = " ".join(data['About'])
result['Education'] = process_key_value(data['Education'], join_sep=';')
result['Language'] = process_key_value(data['LANGUAGE'] if 'LANGUAGE' in data else data['Language'], join_sep=';')
result['Language'] = process_key_value(data['Language'], join_sep=';')
result['Technical Skill'] = process_key_value(data['Technical Skill'], join_sep=';')
result['Experience'] = process_key_value(data['Experience'], join_sep=';')

# Write to CSV
csv_columns = ['Personal detail', 'About', 'Education', 'Language', 'Technical Skill', 'Experience']
with open('cv.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerow(result)

print("CSV file created successfully.")


CSV file created successfully.


In [76]:
pd.read_csv('cv.csv')

,Personal detail,About,Education,Language,Technical Skill,Experience
0,Name:Le Huu Minh;Nationality:Vietnamese;DOB:19...,+ 6+ years of experience in Operation System +...,Now:FPT Polytechnic;Major: Business informatio...,Vietnamese:Native;English:Base,"Domain:Monitoring, Operation System;Monitoring...",FCCL (9/2023 now);Customer:Japanese Customer;D...


In [67]:
import re
import csv

# Mapping for possible section header variations
section_mapping = {
    "PERSONAL DETAIL": "Personal Detail",
    "PERSONAL DETAILS": "Personal Detail",
    "ABOUT": "About",
    "ABOUTS": "About",
    "EDUCATIONAL BACKGROUND": "Educational Background",
    "EDUCATION": "Educational Background",
    "LANGUAGE": "Language",
    # Add more mappings as needed
}

def normalize_section_header(header):
    """
    Normalize a section header using the mapping.
    """
    header_clean = header.strip().upper()
    # Return the normalized name if found; otherwise, capitalize normally.
    return section_mapping.get(header_clean, header_clean.title())

def parse_line(line):
    """
    Split a line into key-value parts using tab or multiple spaces as delimiters.
    """
    if '\t' in line:
        parts = line.split('\t')
    else:
        parts = re.split(r'\s{2,}', line)
    return [part.strip() for part in parts if part.strip()]

# Read the text file containing the CV data.
with open('cv.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

data = {}
current_section = None
section_content = []

for line in lines:
    line = line.strip()
    if not line:
        continue

    # Check if line is a section header.
    # Either it is all uppercase (with spaces) or it matches one of our mapped keys.
    if re.fullmatch(r'[A-Z ]+', line) or line.strip().upper() in section_mapping:
        # Save content of previous section, if any.
        if current_section and section_content:
            if current_section not in data:
                data[current_section] = " ".join(section_content)
            else:
                data[current_section] += " " + " ".join(section_content)
        # Normalize the section header.
        current_section = normalize_section_header(line)
        section_content = []
        continue

    # Try parsing the line as a key-value pair.
    parts = parse_line(line)
    if len(parts) == 2:
        key, value = parts
        if current_section in ["Personal Detail", "Language"]:
            if current_section == "Language":
                if "Language" not in data:
                    data["Language"] = {}
                data["Language"][key] = value
            else:
                data[key] = value
        elif current_section == "Educational Background":
            # Collect multiple education entries in a list.
            if "Education" not in data:
                data["Education"] = []
            data["Education"].append(f"{key}: {value}")
        else:
            # For free text sections like "About"
            section_content.append(line)
    else:
        section_content.append(line)

# Save any trailing section content.
if current_section and section_content:
    if current_section not in data:
        data[current_section] = " ".join(section_content)
    else:
        data[current_section] += " " + " ".join(section_content)

# Flatten the data for CSV writing.
flat_data = {}
for key, value in data.items():
    if isinstance(value, dict):
        for sub_key, sub_value in value.items():
            flat_data[f"{key}_{sub_key}"] = sub_value
    elif isinstance(value, list):
        flat_data[key] = "; ".join(value)
    else:
        flat_data[key] = value

# Write the data to a CSV file.
with open('cv.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=flat_data.keys())
    writer.writeheader()
    writer.writerow(flat_data)

print("CSV file 'cv.csv' created successfully!")


CSV file 'cv.csv' created successfully!


In [66]:
# Path to the input text file and output CSV file
input_folder = 'output'
output_file = 'csv'

# List of section names to look for (these will become CSV columns)
section_names = [
    'PERSONAL DETAIL',
    'ABOUT',
    'EDUCATIONAL BACKGROUND',
    'CERTIFICATION',
    'LANGUAGE',
    'TECHNICAL SKILL',
    'EXPERIENCE'
]
id=0
# Create a CSV file to store all records
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # Write the header (section names)
    writer.writerow(['ID'] + section_names)

    # Process each text file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):  # Only process text files
            input_file = os.path.join(input_folder, filename)
            id=id+1
            # Read the text file and split by section
            with open(input_file, 'r', encoding='utf-8') as file:
                lines = file.readlines()

            # Parse sections
            data = {}
            current_section = None
            for line in lines:
                line = line.strip()
                if line in section_names:
                    current_section = line
                    data[current_section] = []
                elif current_section:
                    data[current_section].append(line)

            # Combine section content into a single string
            for section in data:
                data[section] = ' '.join(data[section]).strip()

            # Create a row with all section data
            row = [id] + [data.get(section, '') for section in section_names]
            writer.writerow(row)

print(f"Structured CSV has been saved to {output_file}")

PermissionError: [Errno 13] Permission denied: 'csv'

tiêu đề đang liền với văn bản